In [28]:
import pandas as pd
from fractions import Fraction

csv_path = "Data/mr_boston_flattened.csv"

In [29]:
cocktail_df = pd.read_csv(csv_path, encoding="utf-8")
cocktail_df.head()

,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,instructions,glass,glass-size
0,Gauguin,Cocktail Classics,2 oz,Light Rum,1 oz,Passion Fruit Syrup,1 oz,Lemon Juice,1 oz,Lime Juice,NaN,NaN,NaN,NaN,Combine ingredients with a cup of crushed ice ...,Old-Fashioned Glass,6 to 8 ounces
1,Fort Lauderdale,Cocktail Classics,1 1/2 oz,Light Rum,1/2 oz,Sweet Vermouth,1/4 oz,Juice of Orange,1/4 oz,Juice of a Lime,NaN,NaN,NaN,NaN,Shake with ice and strain into old-fashioned g...,Old-Fashioned Glass,6 to 8 ounces
2,Apple Pie,Cordials and Liqueurs,3 oz,Apple schnapps,1 oz,Cinnamon schnapps,NaN,Apple slice,NaN,NaN,NaN,NaN,NaN,NaN,Pour into ice-filled old-fashioned glass. Garn...,Old-Fashioned Glass,6 to 8 ounces
3,Cuban Cocktail No. 1,Cocktail Classics,1/2 oz,Juice of a Lime,1/2 oz,Powdered Sugar,2 oz,Light Rum,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces
4,Cool Carlos,Cocktail Classics,1 1/2 oz,Dark rum,2 oz,Cranberry Juice,2 oz,Pineapple Juice,1 oz,Orange curacao,1 oz,Sour Mix,NaN,NaN,"Mix all ingredients except curacao with ice, s...",Collins Glass,14 to 16 ounces


In [30]:
# Create list of measurement columns to loop through
ingredient_cols = ["ingredient-1","ingredient-2","ingredient-3","ingredient-4","ingredient-5","ingredient-6"]
measurement_cols = ["measurement-1", "measurement-2", "measurement-3", "measurement-4", "measurement-5", "measurement-6"]

# Loop through columns and strip units
for col in measurement_cols:
        cocktail_df[col]= cocktail_df[col].str.strip(" oz")
    
cocktail_df

,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,instructions,glass,glass-size
0,Gauguin,Cocktail Classics,2,Light Rum,1,Passion Fruit Syrup,1,Lemon Juice,1,Lime Juice,NaN,NaN,NaN,NaN,Combine ingredients with a cup of crushed ice ...,Old-Fashioned Glass,6 to 8 ounces
1,Fort Lauderdale,Cocktail Classics,1 1/2,Light Rum,1/2,Sweet Vermouth,1/4,Juice of Orange,1/4,Juice of a Lime,NaN,NaN,NaN,NaN,Shake with ice and strain into old-fashioned g...,Old-Fashioned Glass,6 to 8 ounces
2,Apple Pie,Cordials and Liqueurs,3,Apple schnapps,1,Cinnamon schnapps,NaN,Apple slice,NaN,NaN,NaN,NaN,NaN,NaN,Pour into ice-filled old-fashioned glass. Garn...,Old-Fashioned Glass,6 to 8 ounces
3,Cuban Cocktail No. 1,Cocktail Classics,1/2,Juice of a Lime,1/2,Powdered Sugar,2,Light Rum,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces
4,Cool Carlos,Cocktail Classics,1 1/2,Dark rum,2,Cranberry Juice,2,Pineapple Juice,1,Orange curacao,1,Sour Mix,NaN,NaN,"Mix all ingredients except curacao with ice, s...",Collins Glass,14 to 16 ounces
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,Wallis Blue Cocktail,Gin,1,"Lime wedge, superfine sugar",1,Gin,1,Triple Sec,1,Fresh Lime Juice,NaN,NaN,NaN,NaN,Rim old-fashioned glass with lime and sugar. F...,Old-Fashioned Glass,6 to 8 ounces
986,Minnehaha Cocktail,Cocktail Classics,1/4,Juice of Orange,1/2,Dry Vermouth,1/2,Sweet Vermouth,1,Old Mr. Boston Dry Gin,NaN,NaN,NaN,NaN,Shake well with cracked ice and strain into 4 ...,Cocktail Glass,6 or more ounces
987,Wallick Cocktail,Gin,1 1/2,Gin,1 1/2,Dry Vermouth,1,Triple Sec,NaN,NaN,NaN,NaN,NaN,NaN,Stir with ice and strain into chilled cocktail...,Cocktail Glass,6 or more ounces
988,Waikiki Beachcomber,Gin,3/4,Gin,3/4,Triple Sec,1/2,Pineapple Juice,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into chilled cocktai...,Cocktail Glass,6 or more ounces


In [31]:
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        frac_str = frac_str.str.lower()
        if "or" in frac_str:
            nums = frac_str.str.split(' or ')
            return nums[0]
        elif "  " in frac_str:
            frac_str = frac_str.replace("  ", " ")
            print(frac_str)
            value = convert_to_float(frac_str)
            return value
        elif "tsp" in frac_str:
            frac_str = frac_str.strip("tsp")
            print(frac_str)
            value = convert_to_float(frac_str)
            return value * 0.166667
        elif "750-ml" in frac_str:
            frac_str = frac_str.strip("750-ml")
            print(frac_str)
            value = convert_to_float(frac_str)
            return value * 25.3
        elif "750ml" in frac_str:
            frac_str = frac_str.strip("750ml")
            print(frac_str)
            return 25.3
        elif "bottles" in frac_str:
            frac_str = frac_str.strip("bottles")
            print(frac_str)
            value = convert_to_float(frac_str)
            return value * 25.3
        elif "for glass" in frac_str:
            nums = frac_str.split('for glass')
            return 1
        elif "/" in frac_str:
            num, denom = frac_str.split('/')
            try:
                leading, num = num.split(' ')
                whole = float(leading)
            except ValueError:
                whole = 0
            frac = float(num) / float(denom)
            return whole - frac if whole < 0 else whole + frac    
        else:
            print("from else")
            print(frac_str)
            return frac_str

In [32]:
cocktail_df['measurement-1'] = cocktail_df['measurement-1'].fillna(0)

In [33]:
# value = cocktail_df['measurement-1'][0]
# convert_to_float(value)

convert_to_float("from else")

from else
from else


'from else'

In [34]:
for item in cocktail_df['measurement-1']:
#         print(item)
        item = convert_to_float(item)
        print(item)

2.0
1.5
3.0
0.5
1.5
2.0
1.25
2.0
0.5
1.5
1.5
1.5
0.75
0.5
1.0
1.0
1.5
1.0
0.5
0.75
2.5
0.75
1.0
1.5
1.5
0.75
1.0
0.75
1.0
1.5
2.0
2.0
1.5
1.5
0.5
1.0
1.5
1.0
2.0
2.0
0.75
0.25
1.5
1.5
3.0
0.75
2.0
1.5
1.5
1.5
2.0
1.5
2.0
1.5
1.6666666666666665
1.0
1.5
1.5
0.75
2.0
0.5
1.5
0.5
0.5
0.5
1.5
1.5
1.5
2.0
0.75
0.75
2.0
2.0
1.5
1.5
1.5
1.0
1.0
1.5
1.0
1.0
1.0
2.0
2.0
1.5
1.0
0.25
1.0
0.5
2.0
1.5
1.5
0.75
1.0
1.5
2.0
1.0
1.6666666666666665
0.25
1.0
1.75
1.0
1.5
0.5
0.75
1.5
1.0
1.5
1.0
0.75
1.5
0.5
0.5
0.75
1.25
2.0
1.5
1.0
2.0
1.0
1.0
2.0
1.5
1.0
1.5
0.75
0.5
2.0
2.0
1.5
1.5
1.25
1.0
1.0
1.5
2.0
2.0
2.0
0.25
2.0
0.5
2.0
1.5
0.5
1.0
1.5
2.25
1.5
2.0
1.0
2.0
1.0
1.0
2.0
0.5
2.0
0.25
1.0
1.0
1.5
1.0
2.0
5
1.5
1.5
1.5
1.0
1.5
1.5
2.0
2.5
2.0
2.0
2.0
1.75
1.0
2.0
1.0
1.5
1.0
1.0
3.0
1.5
1.5
1.5
2.0
2.0
2.0
2.0
1.0
0.75
1.5
2.0
2.0
1.5
for glass
2.0
0.75
2.0
2.0
1.0
1.0
3.0
2.0
for glass
2.0
0.5
2.0
2.0
2.0
2.0
0.5
2.0
0.75
2.0
2.0
1.0
3.0
0.5
2.0
2.0
1.0
1.0
1.0
1.5
1.0
2.0
1.5
6.0
2.0
1.5
1.5
1.5

In [35]:
cocktail_df[measurements_list].apply(convert_to_float)
cocktail_df.head(10)

KeyError: "None of [Index([                        ('2', '1', '1', '1', nan, nan),\n                     ('1 1/2', '1/2', '1/4', '1/4', nan, nan),\n                               ('3', '1', nan, nan, nan, nan),\n                           ('1/2', '1/2', '2', nan, nan, nan),\n                           ('1 1/2', '2', '2', '1', '1', nan),\n                             ('2', '1', '1/2', '1', '1', nan),\n                       ('1 1/4', '1 1/2', '1', nan, nan, nan),\n                   ('2', '1 1/2', '1 1/2', '1 1/2', nan, nan),\n                            ('1/2', '1/2', '', nan, nan, nan),\n       ('1 1/2', '1/4', '1/4', nan, 'Maraschino cherry', nan),\n       ...\n                           ('1', '1/2', '1/2', '1', nan, nan),\n                             ('1', '1/2', '2', nan, nan, nan),\n                     ('1 1/2', '3/4', '1/2', '1/2', nan, nan),\n                         ('3/4', '3/4', '1', '1/2', '1', nan),\n                       ('3/4', '3/4', '3/4', '3/4', nan, nan),\n        

In [36]:
for col in measurement_cols:
    for index, value in cocktail_df[col].items():
        convert_to_float(value)

1/2 
from else
1 egg
1 
2 
2 
from else
3 slices
from else
6 fresh
from else
splash
from else
3 slices
1 3/4
from else
1 splash
from else
3 slices
from else

from else
ginger ale
from else
16 milk
from else
8 milk
from else
2 dashes
from else
cola
from else
soda water
from else
1c
from else
lime wedge
from else
ginger beer
from else
hot milk
from else
20 cardamom seeds
from else
lemonade
from else
lemon twist
from else
twist of lemon
from else
1 small vanilla ice cream
from else
vanilla ice cream
from else
skewered pineapple wedge, mint sprig, candied ginger slice
from else
64 ginger ale
from else
lemon twist
from else
lemon half-wheel, maraschino cherry
from else
1 juice of a lemon
from else
lime wedge
from else
lemon-lime soda
from else
lemon wheel
from else
soda water
from else
generous amount of fresh seasonal fruit
from else
lime zest spiral
from else
soda water
from else
lemon-lime soda
from else
lemon twist
from else
freshly grated nutmeg
from else
ginger ale
from else
lime wedg

1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp gr

1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp grenadine
1/2 tsp gr

RecursionError: maximum recursion depth exceeded in comparison

In [37]:

cocktail_df.head()

,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,instructions,glass,glass-size
0,Gauguin,Cocktail Classics,2,Light Rum,1,Passion Fruit Syrup,1,Lemon Juice,1,Lime Juice,NaN,NaN,NaN,NaN,Combine ingredients with a cup of crushed ice ...,Old-Fashioned Glass,6 to 8 ounces
1,Fort Lauderdale,Cocktail Classics,1 1/2,Light Rum,1/2,Sweet Vermouth,1/4,Juice of Orange,1/4,Juice of a Lime,NaN,NaN,NaN,NaN,Shake with ice and strain into old-fashioned g...,Old-Fashioned Glass,6 to 8 ounces
2,Apple Pie,Cordials and Liqueurs,3,Apple schnapps,1,Cinnamon schnapps,NaN,Apple slice,NaN,NaN,NaN,NaN,NaN,NaN,Pour into ice-filled old-fashioned glass. Garn...,Old-Fashioned Glass,6 to 8 ounces
3,Cuban Cocktail No. 1,Cocktail Classics,1/2,Juice of a Lime,1/2,Powdered Sugar,2,Light Rum,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces
4,Cool Carlos,Cocktail Classics,1 1/2,Dark rum,2,Cranberry Juice,2,Pineapple Juice,1,Orange curacao,1,Sour Mix,NaN,NaN,"Mix all ingredients except curacao with ice, s...",Collins Glass,14 to 16 ounces


In [26]:
measurements_list = cocktail_df[measurement_cols].values.tolist()
cocktail_df["all_measurements"] = measurements_list

ingredients_list = cocktail_df[ingredient_cols].values.tolist()
cocktail_df["all_ingredients"] = ingredients_list

cocktail_df.head()

,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,instructions,glass,glass-size,all_ingredients,all_measurements
0,Gauguin,Cocktail Classics,2,Light Rum,1,Passion Fruit Syrup,1,Lemon Juice,1,Lime Juice,NaN,NaN,NaN,NaN,Combine ingredients with a cup of crushed ice ...,Old-Fashioned Glass,6 to 8 ounces,"[ Light Rum, Passion Fruit Syrup, Lemon Juic...","[2, 1, 1, 1, nan, nan]"
1,Fort Lauderdale,Cocktail Classics,1 1/2,Light Rum,1/2,Sweet Vermouth,1/4,Juice of Orange,1/4,Juice of a Lime,NaN,NaN,NaN,NaN,Shake with ice and strain into old-fashioned g...,Old-Fashioned Glass,6 to 8 ounces,"[ Light Rum, Sweet Vermouth, Juice of Orange...","[1 1/2, 1/2, 1/4, 1/4, nan, nan]"
2,Apple Pie,Cordials and Liqueurs,3,Apple schnapps,1,Cinnamon schnapps,NaN,Apple slice,NaN,NaN,NaN,NaN,NaN,NaN,Pour into ice-filled old-fashioned glass. Garn...,Old-Fashioned Glass,6 to 8 ounces,"[ Apple schnapps, Cinnamon schnapps, Apple s...","[3, 1, nan, nan, nan, nan]"
3,Cuban Cocktail No. 1,Cocktail Classics,1/2,Juice of a Lime,1/2,Powdered Sugar,2,Light Rum,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces,"[ Juice of a Lime, Powdered Sugar, Light Rum...","[1/2, 1/2, 2, nan, nan, nan]"
4,Cool Carlos,Cocktail Classics,1 1/2,Dark rum,2,Cranberry Juice,2,Pineapple Juice,1,Orange curacao,1,Sour Mix,NaN,NaN,"Mix all ingredients except curacao with ice, s...",Collins Glass,14 to 16 ounces,"[ Dark rum, Cranberry Juice, Pineapple Juice...","[1 1/2, 2, 2, 1, 1, nan]"
